In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
df = pd.read_csv('/kaggle/input/zomato-bangalore-restaurants/zomato.csv')

What's the size of our data??

In [ ]:
rows = df.shape[0]
columns = df.shape[1]
print("Rows : "+str(rows)+" Columns :  "+str(columns))

51k entries!!!, banglore is really a foody place.Let's have a look at the dataset

In [ ]:
df.head()

Wait a sec, if there are 51k entries. Does it really mean that there are these many unique places? or are there some redundant branches

In [ ]:
df['name'].value_counts()[:10]

As I thought, there are some redundant names, but are they all same these are just their branches, let's figure it out

In [ ]:
df[df['name'] == 'Petoo'][:5]

...And they are different, in fact at different location with different phone numbers and different ratings. So they are just branches of the company. But wait a sec, can I get an overall idea of the popularity of a restaurant just by knowing the branches of the restaurants

In [ ]:
plt.figure(figsize=(7,7))
df_temp = df['name'].value_counts()[:10]
import seaborn as sns
import matplotlib.pyplot as plt
sns.barplot(x = df_temp , y = df_temp.index)

That was an interesting insgiht, but wait, what do these restaurants do that makes them so popular. Time to figure this out!!

In [ ]:
df.columns

Let's compare one thing, do these restaurant provide online ordering.

These was just the overall ordering example

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(5,5))
df_temp = df['online_order'].value_counts()
plt.title('Overall Online Order Service')
plt.pie(df_temp, labels=df_temp.index)
plt.show()

Now, let's get on with the IVY league restaurants, what I meant to say is the ones which are really popular

In [ ]:
plt.figure(figsize=(5,5))
popular_restaurant = df['name'].value_counts()[:10].index
df_popular = df[df['name'].isin(popular_restaurant)]
df_temp = df_popular['online_order'].value_counts()
plt.pie(df_temp, labels=df_temp.index)
plt.title('Top 10 restaurants')
plt.show()

Whoooho, That's why they are popular. Majority of them are providing options of Online Orders. I think I got the trick to start my own restaurants.

In [ ]:
df.columns

Let's have a look at the ratings, if we can get any interesting gossip it as well.

In [ ]:
df['rate'][:10]

We need to do something regarding the formatting. Let's just make a new column and then sort them to see the top rates restaurants!!

In [ ]:
#There are some null entries, let's handle it first!
df['rate'].isna().sum()

7775!!, let's just them with 0!

In [ ]:
df['rate'] = df['rate'].fillna('0')

In [ ]:
df['rate'].isna().sum()

Well now that we dont have any null entries, it's great time to replace them with integers and get the top rated restaurants

In [ ]:
rating = list(df['rate'])
rating = [i.split('/')[0] for i in rating]
total_length = len(rating)
for i in range(total_length):
    if(rating[i] == 'NEW' or rating[i] == '-'):
        rating[i] = '0'
rating= [ float(x) for x in rating ]

In [ ]:
df['final_ratings'] = rating

In [ ]:
df.head()

In [ ]:
df_temp = df[['name','final_ratings']]

In [ ]:
df_temp.sort_values('final_ratings',ascending = False)[:10]

Some restaurants are really dominating, let's fetch see some more!

In [ ]:
df_temp = df_temp.drop_duplicates()
df_temp.sort_values('final_ratings',ascending = False)[:15]

In [ ]:
sns.distplot(df_temp['final_ratings'],bins = 30)

In [ ]:
df.columns

What's the rest type?, let's figure it out!

In [ ]:
df['rest_type'].unique()

trust me, I have never seen these many types of foods. Our task is to find out which is the most eaten or popular type of food

In [ ]:
plt.figure(figsize=(7,7))
df_temp = df['rest_type'].value_counts()[:10]
sns.barplot(x = df_temp,y = df_temp.index)
plt.title('Busy developers!')

As expected from banglore!

In [ ]:
df.columns

Let's co-relate rest_type and location to figure out which is the most popular food type in which part of banglore!!!

In [ ]:
plt.figure(figsize=(7,7))
rest=df['location'].value_counts()[:10]
sns.barplot(rest,rest.index)

In [ ]:
areas = []
cuisines = []
areas = list(df['location'].value_counts()[:10].index)
for i in areas:
    df_temp = df[df['location'] == i]
    cuisines.append((str(df_temp['cuisines'].value_counts()[:1].index[0])))

df_temp =pd.DataFrame()
df_temp['areas'] = areas
df_temp['food'] = cuisines
df_temp


Done!!

In [ ]:
df.columns

So what is dish_liked? , let's figure it out

In [ ]:
df.head()

DISH_LIKED

In [ ]:
df_temp = df[df['rest_type'] == 'Casual Dining']
df_temp.head()

In [ ]:
words = list(df_temp['dish_liked'])
word_cloud = []
for i in words:
    if(type(i) == str):
        temp = i.split(',')
        for i in temp:
            i=" ".join(i.split())
            word_cloud.append(i)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
#convert list to string and generate
unique_string=(" ").join(word_cloud)
wordcloud = WordCloud(width = 1000, height = 500).generate(unique_string)
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.savefig("your_file_name"+".png", bbox_inches='tight')
plt.show()
plt.close()

Let's Generalize it for all the types of food types

In [ ]:
'''
'Quick Bites',
 'Casual Dining',
 'Cafe',
 'Delivery',
 'Dessert Parlor',
 'Takeaway, Delivery',
 'Casual Dining, Bar',
 'Bakery',
 'Beverage Shop',
 'Bar'
'''
import matplotlib.pyplot as plt
rest_type = list(df['rest_type'].value_counts()[:10].index)
for i in rest_type:
    
    df_temp = df[df['rest_type'] == i]
    words = list(df_temp['dish_liked'])
    word_cloud = []
    for i in words:
        if(type(i) == str):
            temp = i.split(',')
            for i in temp:
                i=" ".join(i.split())
                word_cloud.append(i)
    unique_string=(" ").join(word_cloud)
    wordcloud = WordCloud(width = 1000, height = 500).generate(unique_string)
    plt.figure(figsize=(15,8))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.savefig("your_file_name"+".png", bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
rest_type = list(df['rest_type'].unique())
len(rest_type)

94 is too much, let's just take the top 10

In [ ]:
df.columns

Last part, and I think the most difficult one is reviews_list, let's defeat this one and then we are done!

In [ ]:
df['reviews_list'].head()

In [ ]:
# Code is referenced from SHAHULES Kaggle Notebook
from tqdm import tqdm
all_ratings = []

for name,ratings in tqdm(zip(df['name'],df['reviews_list'])):
    ratings = eval(ratings)
    for score, doc in ratings:
        if score:
            score = score.strip("Rated").strip()
            doc = doc.strip('RATED').strip()
            score = float(score)
            all_ratings.append([name,score, doc])

In [ ]:
all_ratings

In [ ]:
rating_df=pd.DataFrame(all_ratings,columns=['name','rating','review'])

In [ ]:
rating_df[:10]

Thank you SHAHULES for providing notebook, let's see if I can improvised your code.We just learned to visualize the data using wordcloud. Why not try it here!

In [ ]:
rating_df['name'].value_counts()[:10].index

In [ ]:
import matplotlib.pyplot as plt
rest_type = list(rating_df['name'].value_counts()[:1].index)
reviews = []
wordcloud = []
for i in rest_type:
    df_temp = rating_df[rating_df['name'] == i]
    reviews = list(df_temp['review'])

print(len(reviews))

for i in reviews:
    temp = i.split(' ')
    for i in temp:
        wordcloud.append(i)
print(len(wordcloud))

print(wordcloud)
    

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
temp = []
for i in wordcloud:
    if(i not in stop):
        temp.append(i)
len(temp)

In [ ]:
unique_string=(" ").join(temp)
wordcloud = WordCloud(width = 1000, height = 500).generate(unique_string)
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.savefig("your_file_name"+".png", bbox_inches='tight')
plt.show()
plt.close()

Let's generalize it!

In [ ]:
''' 
'Hammered', 'Mast Kalandar', 'Truffles', 'Onesta', 'Crawl Street',
       'Brooks and Bonds Brewery', 'Stoner', 'Cafe Azzure', 'Cafe @ Elanza',
       'Smally's Resto Cafe'],
'''
import matplotlib.pyplot as plt
rest_type = list(rating_df['name'].value_counts()[:10].index)
for i in rest_type:
    reviews = []
    wordcloud = []

    df_temp = rating_df[rating_df['name'] == i]
    reviews = list(df_temp['review'])
    for i in reviews:
        temp = i.split(' ')
        for i in temp:
            wordcloud.append(i)
    temp = []
    for i in wordcloud:
        if(i not in stop):
            temp.append(i)
    unique_string=(" ").join(temp)
    wordcloud = WordCloud(width = 1000, height = 500).generate(unique_string)
    plt.figure(figsize=(15,8))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.savefig("your_file_name"+".png", bbox_inches='tight')
    plt.show()
    plt.close()
    
        
    
    